# Learning Goals

By the end of this tutorial you will: <br>
* Understand how to use source-clustering for searching data.
* Learn to search a survey for observations of a known target.
* Be able to plot an observation.
* Make your own comparisons and conclusions based on the target.


# Introduction

Ultra-faint dwarf (UFD) galaxies are a subset of dwarf galaxies that are classified by having one hundred thousand to a few hundred thousand stars, making them the faintest galaxies in the universe. They differ from globular clusters in that they have a significant amount of dark matter which extends beyond the visible stellar matter. Because of their high concentration of dark matter, astronomers believe there could be unique insights into the early universe in UFD's. All UFD's that have been discovered are ancient systems that are believed to have formed only a few million years after the Big Bang.  

The target we will focus on in this tutorial is Pegasus V whose discovery was recently published in the paper "Pegasus V -- a newly discovered ultra-faint dwarf galaxy on the outskirts of Andromeda" by Collins et al. 2022. 


# Imports

The following cell holds the imported packages. These packages are necessary for running the rest of the cells in this notebook. A description of each import is as follows:

* numpy to handle array functions
* pandas to handle date conversions
* fits from astropy.io for accessing FITS files
* Table from astropy.table for creating tidy tables of the data
* matplotlib.pyplot for plotting data
* Mast and Observations from astroquery.mast for querying data and observations from the MAST archive


In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
from astroquery.mast import Mast
from astroquery.mast import Observations
from astroquery.mast import Catalogs

# Main Content

# Search PanSTARRS for Pegasus V

## What is source-clustering?

# Plot Something (tbd)

# Exercises

# Additional Resources

You can find the discovery paper of this target here: https://arxiv.org/abs/2204.09068 

# About this Notebook

**Author:** Emma Lieb <br>
**Last Updated:** Jul 11th 2022

# Citations

* [Citation for `astropy`](https://www.astropy.org/acknowledging.html)